# Automobile Accidents
- `accidentsFull.csv` 파일에는 세 단계 부상 수준(NO INJURY, INJURY, FATALITY)으로 기록된 2001년 미국의 실제 자동차 사고 42,183건에 대한 정보가 담겨 있다. 각 사고에 대해 요일, 기상 조건, 도로 종류와 같은 추가 정보도 기록되어 있다. 어떤 회사가 초기 보고서와 이 시스템의 연관된 데이터(그중 일부는 GPS-지원 보고에 의존함)에 근거하여 사고의 심각성을 신속하게 분류하는 시스템을 개발하고자 한다.
- 목적은 보고된 사고에 부상이 동반될지(MAX_SEV_IR = 1 또는 2), 부상이 없을지(MAX_SEV_IR = 0)을 예측하는 것이다. 이를 위해 MAX_SEV_IR = 1 또는 2이면 값이 "Yes"가 되고 그렇지 않으면 "No"가 되는 INJURY라는 가변수를 생성하시오.

In [1]:
!pip install dmba

In [2]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import dmba

import matplotlib.pylab as plt

no display found. Using non-interactive Agg backend


## Load the data set

In [3]:
accident_df = dmba.load_data('accidentsFull.csv')

Determine the shape of the data frame. It has 42183 rows and 24 columns

In [4]:
accident_df.shape

(42183, 24)

Show the top rows of the dataframe

In [5]:
accident_df.head()

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,SUR_COND,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR
0,0,2,2,1,0,1,0,3,0,0,...,4,0,3,1,1,1,1,0,0,1
1,1,2,1,0,0,1,1,3,2,0,...,4,0,3,2,2,0,0,1,0,0
2,1,2,1,0,0,1,0,3,2,0,...,4,1,2,2,2,0,0,1,0,0
3,1,2,1,1,0,0,0,3,2,0,...,4,1,2,2,1,0,0,1,0,0
4,1,1,1,0,0,1,0,3,2,0,...,4,0,2,3,1,0,0,1,0,0


In [6]:
accident_df.columns

Index(['HOUR_I_R', 'ALCHL_I', 'ALIGN_I', 'STRATUM_R', 'WRK_ZONE', 'WKDY_I_R',
       'INT_HWY', 'LGTCON_I_R', 'MANCOL_I_R', 'PED_ACC_R', 'RELJCT_I_R',
       'REL_RWY_R', 'PROFIL_I_R', 'SPD_LIM', 'SUR_COND', 'TRAF_CON_R',
       'TRAF_WAY', 'VEH_INVL', 'WEATHER_R', 'INJURY_CRASH', 'NO_INJ_I',
       'PRPTYDMG_CRASH', 'FATALITIES', 'MAX_SEV_IR'],
      dtype='object')

In [7]:
accident_df.describe()

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,SUR_COND,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR
count,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,...,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000
mean,0.429344,1.912832,1.131546,0.491620,0.022616,0.771614,0.150321,1.492521,1.337079,0.040514,...,1.290710,0.516322,1.477491,1.816964,1.142783,0.497736,0.778702,0.491217,0.011047,0.519830
std,0.494988,0.282084,0.338000,0.499936,0.148677,0.419797,0.418952,0.789874,0.929756,0.197164,...,0.780524,0.749417,0.584851,0.684843,0.349855,0.500001,1.035169,0.499929,0.104524,0.521256
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,2.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,0.000000,...,1.000000,0.000000,1.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,1.000000,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,2.000000,2.000000,0.000000,...,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,9.000000,3.000000,2.000000,1.000000,...,9.000000,2.000000,3.000000,23.000000,2.000000,1.000000,31.000000,1.000000,1.000000,2.000000


In [8]:
accident_df['SPD_LIM'].describe()

count    42183.000000
mean        43.547875
std         12.948396
min          5.000000
25%         35.000000
50%         40.000000
75%         55.000000
max         75.000000
Name: SPD_LIM, dtype: float64

In [9]:
print("Number of rows\n", accident_df.count())
reduce_df = accident_df.dropna()
print("\nNumber of rows after removing rows with missing values\n", reduce_df.count())

Number of rows
 HOUR_I_R          42183
ALCHL_I           42183
ALIGN_I           42183
STRATUM_R         42183
WRK_ZONE          42183
WKDY_I_R          42183
INT_HWY           42183
LGTCON_I_R        42183
MANCOL_I_R        42183
PED_ACC_R         42183
RELJCT_I_R        42183
REL_RWY_R         42183
PROFIL_I_R        42183
SPD_LIM           42183
SUR_COND          42183
TRAF_CON_R        42183
TRAF_WAY          42183
VEH_INVL          42183
WEATHER_R         42183
INJURY_CRASH      42183
NO_INJ_I          42183
PRPTYDMG_CRASH    42183
FATALITIES        42183
MAX_SEV_IR        42183
dtype: int64

Number of rows after removing rows with missing values
 HOUR_I_R          42183
ALCHL_I           42183
ALIGN_I           42183
STRATUM_R         42183
WRK_ZONE          42183
WKDY_I_R          42183
INT_HWY           42183
LGTCON_I_R        42183
MANCOL_I_R        42183
PED_ACC_R         42183
RELJCT_I_R        42183
REL_RWY_R         42183
PROFIL_I_R        42183
SPD_LIM           42183
SU

In [10]:
accident_df.dtypes

HOUR_I_R          int64
ALCHL_I           int64
ALIGN_I           int64
STRATUM_R         int64
WRK_ZONE          int64
WKDY_I_R          int64
INT_HWY           int64
LGTCON_I_R        int64
MANCOL_I_R        int64
PED_ACC_R         int64
RELJCT_I_R        int64
REL_RWY_R         int64
PROFIL_I_R        int64
SPD_LIM           int64
SUR_COND          int64
TRAF_CON_R        int64
TRAF_WAY          int64
VEH_INVL          int64
WEATHER_R         int64
INJURY_CRASH      int64
NO_INJ_I          int64
PRPTYDMG_CRASH    int64
FATALITIES        int64
MAX_SEV_IR        int64
dtype: object

보고된 사고에 부상이 동반될지(MAX_SEV_IR = 1 OR 2), 부상이 없을지(MAX_SEV_IR = 0)를 예측하기 위해 MAX_SEV_IR = 1 또는 2이면 값이 "Yes"가 되고 그렇지 않으면 "No"가 되는 `INJURY`라는 가변수를 생성.

In [11]:
accident_df['INJURY'] = accident_df['MAX_SEV_IR'].apply(lambda x: "Yes" if x > 0 else "No")
accident_df.head()

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR,INJURY
0,0,2,2,1,0,1,0,3,0,0,...,0,3,1,1,1,1,0,0,1,Yes
1,1,2,1,0,0,1,1,3,2,0,...,0,3,2,2,0,0,1,0,0,No
2,1,2,1,0,0,1,0,3,2,0,...,1,2,2,2,0,0,1,0,0,No
3,1,2,1,1,0,0,0,3,2,0,...,1,2,2,1,0,0,1,0,0,No
4,1,1,1,0,0,1,0,3,2,0,...,0,2,3,1,0,0,1,0,0,No


## Part a

이 데이터셋의 정보를 사용하여, 막 보고된 사고에 추가 정보가 없다면 예측은 무엇이 되어야 하는가?(INJURY = Yes 또는 No?) 그 이유는 무엇인가?

In [12]:
# create a value count table based on 'INJURY'
injury_value_cnt = accident_df['INJURY'].value_counts()
print(injury_value_cnt)

INJURY
Yes    21462
No     20721
Name: count, dtype: int64


In [13]:
from pandas.api.types import CategoricalDtype

# calculate probability of injury
inj_prob = "{:.2%}".format(
    injury_value_cnt["Yes"] / 
    (injury_value_cnt["Yes"] + injury_value_cnt["No"])
    )
print("probability of injury: ",inj_prob)

probability of injury:  50.88%


### Ans)

**주어진 데이터 세트의 사고 중 약 51%(50.88%)가 부상으로 이어졌으므로, 사고가 발생하면 부상으로 이어질 가능성이 조금 더 높다고 예측해야 합니다.**

## Part b

이 데이터셋의 처음 12개 레코드를 선택하여 응답값(INJURY)과 2개의 예측 변수 WEATHER_R과 TRAF_CON_R만 고려하자.

In [14]:
# Convert all columns to categorical type
for col in accident_df.columns:
    accident_df[col] = accident_df[col].astype('category')
    
# Create a new subset with only the required records
new_df = accident_df.loc[0:11, ["INJURY", "WEATHER_R", "TRAF_CON_R"]]
new_df

,INJURY,WEATHER_R,TRAF_CON_R
0,Yes,1,0
1,No,2,0
2,No,2,1
3,No,1,1
4,No,1,0
5,Yes,2,0
6,No,2,0
7,Yes,1,0
8,No,2,0
9,No,2,0


### i.
12개의 레코드들에 대해 두 예측 변수의 함수로서 INJURY를 검토하는 피벗 테이블을 작성하십시오. 피벗 테이블에서 이 세 변수를 모두 행/열로 사용하시오.

In [15]:
!pip install pivottablejs

In [16]:
import pivottablejs

# Display the pivot table for the new_df
pivottablejs.pivot_ui(new_df)

### ii.
예측 변수들의 가능한 조합 6가지에 대해, 부상이 있을(INJURY = Yes) 정확한 베이즈 조건부 확률(Exact Bayes conditional probability)을 계산하시오.

In [17]:
# Calculating probabilities

# P(Injury=yes|WEATHER_R = 1, TRAF_CON_R =0)
numerator1 = 2/3 * 3/12
denominator1 = 3/12
prob1 = numerator1 / denominator1

# P(Injury=yes|WEATHER_R = 1, TRAF_CON_R =1)
numerator2 = 0 * 3/12
denominator2 = 1/12
prob2 = numerator2 / denominator2 if denominator2 != 0 else np.nan

# P(Injury=yes| WEATHER_R = 1, TRAF_CON_R =2)
numerator3 = 0 * 3/12
denominator3 = 1/12
prob3 = numerator3 / denominator3 if denominator3 != 0 else np.nan

# P(Injury=yes| WEATHER_R = 2, TRAF_CON_R =0)
numerator4 = 1/3 * 3/12
denominator4 = 6/12
prob4 = numerator4 / denominator4

# P(Injury=yes| WEATHER_R = 2, TRAF_CON_R =1)
numerator5 = 0 * 3/12
denominator5 = 1/12
prob5 = numerator5 / denominator5 if denominator5 != 0 else np.nan

# P(Injury=yes| WEATHER_R = 2, TRAF_CON_R =2)
numerator6 = 0 * 3/12
denominator6 = 0
prob6 = numerator6 / denominator6 if denominator6 != 0 else np.nan

# Creating DataFrame
options = [1, 2, 3, 4, 5, 6]
probabilities = [prob1, prob2, prob3, prob4, prob5, prob6]
prob_df = pd.DataFrame({'Option #': options, 'Probability': probabilities})

# Rounding probabilities to 3 decimal places
prob_df = prob_df.round({'Probability': 3})
prob_df


,Option #,Probability
0,1,0.667
1,2,0.000
2,3,0.000
3,4,0.167
4,5,0.000
5,6,NaN


### iii.
이 확률값들과 컷오프 값 0.5를 사용해 이 12개의 사고를 분류하시오.

In [18]:
# Add probability results to the new_df
new_df_prob = new_df.copy()

# Display the head of the updated DataFrame
print(new_df_prob.head())

new_df_prob

  INJURY WEATHER_R TRAF_CON_R
0    Yes         1          0
1     No         2          0
2     No         2          1
3     No         1          1
4     No         1          0


,INJURY,WEATHER_R,TRAF_CON_R
0,Yes,1,0
1,No,2,0
2,No,2,1
3,No,1,1
4,No,1,0
5,Yes,2,0
6,No,2,0
7,Yes,1,0
8,No,2,0
9,No,2,0


In [19]:
# Probability of injury
prob_inj = [0.667,
0.167,
0,
0,
0.667, 
0.167, 
0.167, 
0.667, 
0.167, 
0.167, 
0.167, 
0]

# Add the PROB_INJURY column to the new_df_prob DataFrame
new_df_prob['PROB_INJURY'] = prob_inj

# Add a column for injury prediction based on a cutoff of 0.5
new_df_prob['PREDICT_PROB'] = np.where(
    new_df_prob['PROB_INJURY'] > 0.5, "yes", "no"
    )

# Display the updated DataFrame
new_df_prob


,INJURY,WEATHER_R,TRAF_CON_R,PROB_INJURY,PREDICT_PROB
0,Yes,1,0,0.667,yes
1,No,2,0,0.167,no
2,No,2,1,0.000,no
3,No,1,1,0.000,no
4,No,1,0,0.667,yes
5,Yes,2,0,0.167,no
6,No,2,0,0.167,no
7,Yes,1,0,0.667,yes
8,No,2,0,0.167,no
9,No,2,0,0.167,no


### iv.
WEATHER_R = 1과 TRAF_CON_R = 1이 주어졌을 때, 부상이 있을 나이브 베이즈 조건부 확률을 손으로 계산하시오.

In [20]:
# P(INJURY=Yes| WEATHER_R = 1, TRAF_CON_R = 1):
# 사고에서 부상의 확률
# = (INJURY=Yes인 경우 WEATHER_R = 1의 비율) 
#    * (INJURY=Yes인 경우 TRAF_CON_R = 1의 비율)
#    * (모든 경우에서 INJURY=yes의 비율)
prob = 2/3 * 0/3 * 3/12
prob

0.0

### v.
scikit-learn을 사용해 2개의 예측 변수를 갖는 12개의 레코드들에 대해 나이브 베이즈 분류기를 실행하시오. 12개 레코드 모두에 대해 확률값과 분류 결과를 구해서 모델의 출력값을 살펴보시오. 이 출력값을 정확한 베이즈 분류 결과와 비교하시오. 분류 결과가 동일한가? 관측치들의 랭킹(=순위)은 동일한가?

In [21]:
from sklearn.naive_bayes import CategoricalNB, MultinomialNB
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score


In [22]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message="divide by zero encountered in log")

# 데이터 검증 및 전처리
new_df = new_df.copy()

# 예측 변수(X)와 타겟 변수(y) 설정
X = new_df[["WEATHER_R", "TRAF_CON_R"]]
y = new_df['INJURY'].astype('category')

# Naive Bayes 모델 생성 및 교차 검증
nb_model = CategoricalNB(alpha=0, fit_prior=False, force_alpha=True)

# 모델 훈련 및 예측
nb_model.fit(X, y)
pred_probs = np.round(nb_model.predict_proba(X), 4)
pred_classes = nb_model.predict(X)

print("## $class")
print("##   ", "    ".join(map(str, range(1, len(pred_classes) + 1))))  # 인덱스 번호
print("##   ", "   ".join(pred_classes))  # yes/no로 변환한 클래스
print("##   Levels: no yes")  # 클래스 레벨
print("##")
print("## posterior")
posterior_df = pd.DataFrame(pred_probs, columns=["no", "yes"])
posterior_df.index += 1  # 인덱스를 1부터 시작하도록 조정
print(posterior_df)

# 혼동 행렬과 Kappa 점수 계산
conf_matrix = confusion_matrix(y, pred_classes)

accuracy = round(accuracy_score(y, pred_classes), 4)
kappa_score = round(cohen_kappa_score(y, pred_classes), 4)
print("\nConfusion Matrix:\n", conf_matrix)

print()
print("##   usekernel  Accuracy   Kappa")
print("##   FALSE     ", accuracy, " ", kappa_score)
print("##    TRUE     ", accuracy, " ", kappa_score)

# 새로운 데이터프레임에 예측 결과 추가
new_df["PROB_INJURY_YES"] = pred_probs[:, 1]
new_df["PREDICTED_CLASS"] = pred_classes

print()
print("Exact Bayes conditional probability\n", new_df_prob)
print()
print("scikit_learn CategoricalNB:\n", new_df)

## $class
##    1    2    3    4    5    6    7    8    9    10    11    12
##    Yes   No   No   No   Yes   No   No   Yes   No   No   No   No
##   Levels: no yes
##
## posterior
        no     yes
1   0.2500  0.7500
2   0.5714  0.4286
3   1.0000  0.0000
4   1.0000  0.0000
5   0.2500  0.7500
6   0.5714  0.4286
7   0.5714  0.4286
8   0.2500  0.7500
9   0.5714  0.4286
10  0.5714  0.4286
11  0.5714  0.4286
12  1.0000  0.0000

Confusion Matrix:
 [[8 1]
 [1 2]]

##   usekernel  Accuracy   Kappa
##   FALSE      0.8333   0.5556
##    TRUE      0.8333   0.5556

Exact Bayes conditional probability
    INJURY WEATHER_R TRAF_CON_R  PROB_INJURY PREDICT_PROB
0     Yes         1          0        0.667          yes
1      No         2          0        0.167           no
2      No         2          1        0.000           no
3      No         1          1        0.000           no
4      No         1          0        0.667          yes
5     Yes         2          0        0.167           no
6   

### 비교 분석
모델의 분류 결과와 정확한 베이즈 조건부 확률 결과를 비교하여 다음과 같은 결론을 도출할 수 있습니다:

1. **분류 결과 비교**: 
   - `PREDICT_PROB` (scikit-learn CategoricalNB)와 `PROB_INJURY` (정확한 베이즈)의 예측 결과는 항상 일치했습니다.
   - 그러나 확률 값에는 차이가 존재했습니다. 예를 들어, 1번 레코드는 두 모델 모두 `yes`로 예측되었지만, 그 확률은 각각 `0.7500`과 `0.667`로 서로 달랐습니다.
   - 이러한 차이는 scikit-learn에서 정의된 CategoricalNB 모델의 학습 방법이 정확한 베이즈 모델과 다르기 때문입니다. CategoricalNB는 Laplace smoothing과 같은 기법을 사용하여 확률을 계산하고, 데이터의 빈도수에 따라 확률을 조정합니다. 반면, 정확한 베이즈 모델은 주어진 데이터에 대한 확률을 직접적으로 계산하여 확률값을 제공합니다.

2. **관측치 랭킹 비교**:
   - `PROB_INJURY`와 `PREDICT_PROB`의 값에 따라 관측치의 랭킹이 동일한지 확인할 수 있습니다. 
   - 일반적으로 확률값에 기반한 랭킹이 일치하므로, 각 레코드의 예측이 어떻게 정렬되는지 확인해야 합니다. 

결론적으로, 모델의 출력값과 정확한 베이즈 분류 결과는 예측 클래스에서 일치하지만, 예측 확률에서는 차이가 존재했습니다. 이는 CategoricalNB의 학습 방법과 정확한 베이즈 모델 간의 차이에 기인합니다. 각 모델의 성능을 비교하고 이해하는 것은 예측 문제를 해결하는 데 중요한 요소입니다.

# C
이제 전체 데이터셋으로 돌아가보자. 데이터를 학습 데이터셋(60%)과 검증 데이터셋(40%)으로 분할하시오.

In [23]:
np.random.seed(22)

# 인덱스를 60% 비율로 샘플링
train_index = np.random.choice(
    accident_df.index, 
    size=int(len(accident_df) * 0.6),
    replace=False
    )
train_df = accident_df.loc[train_index]
valid_df = accident_df.drop(train_index)

In [24]:
print(train_df.head(), "\n Train DF shape: ", train_df.shape)
print(valid_df.head(), "\n Valid DF shape: ", valid_df.shape)

      HOUR_I_R ALCHL_I ALIGN_I STRATUM_R WRK_ZONE WKDY_I_R INT_HWY LGTCON_I_R  \
30123        0       2       1         0        0        0       0          1   
32880        1       2       1         0        0        1       0          1   
29404        1       2       1         0        0        0       1          1   
12944        1       2       2         0        0        0       0          1   
23849        0       2       1         0        0        1       0          1   

      MANCOL_I_R PED_ACC_R  ... TRAF_CON_R TRAF_WAY VEH_INVL WEATHER_R  \
30123          2         0  ...          2        2        2         1   
32880          2         0  ...          0        1        2         1   
29404          2         0  ...          0        2        2         1   
12944          0         0  ...          2        1        1         2   
23849          2         0  ...          1        3        2         1   

      INJURY_CRASH NO_INJ_I PRPTYDMG_CRASH FATALITIES MAX_SEV_IR INJ

## i.
예측하는 시점에 사고 자체에 대한 정보나 초기 보고서가 가용하지 않다고 가정할 때(위치 특성과 기상 조건 등만 주어짐), 어떤 예측 변수들을 분석에 포함시킬 수 있는가?(파일의 데이터 명세 페이지를 참조하시오.)

### Ans)
사고 예측 모델을 구축하기 위해, 다음의 변수(표에서 파란색 표시)를 선택하였습니다. 이 변수들은 주어진 시간, 위치 특성, 기상 조건 등의 정보와 함께 사용될 수 있습니다.

| 변수명           | 설명                                                                                                         |
|------------------|-------------------------------------------------------------------------------------------------------------|
| <span style="color:blue">**HOUR_I_R**</span>         | 출퇴근 시간 여부 (1=출퇴근 시간, 0=아님)                                                                     |
| ALCOHOL_I        | 음주 관련 여부 (1=음주 관련, 2=관련 없음)                                                                   |
| <span style="color:blue">**ALIGN_I**</span>          | 도로 형태 (1=직선, 2=커브)                                                                                  |
| STRATUM_R        | NASS에 의해 분류된 승용차 또는 경트럭 사고 여부 (1=해당, 0=해당 없음)                                       |
| <span style="color:blue">**WRK_ZONE**</span>         | 공사 구역 여부 (1=예, 0=아니오)                                                                             |
| <span style="color:blue">**WKDY_I_R**</span>         | 주중 여부 (1=주중, 0=주말)                                                                                  |
| <span style="color:blue">**INT_HWY**</span>          | 고속도로 여부 (1=예, 0=아니오)                                                                             |
| <span style="color:blue">**LGTCON_I_R**</span>       | 조명 상태 (1=낮, 2=어두움 (황혼 포함), 3=어두우나 조명 있음, 4=황혼)                                        |
| MAN_COL_I        | 사고 유형 (0=충돌 없음, 1=정면충돌, 2=기타 충돌)                                                            |
| PED_ACC_R        | 보행자/자전거 연관 여부 (1=연관 있음, 0=없음)                                                               |
| REL_JCT_I_R      | 교차로 사고 여부 (1=교차로에서 발생, 0=아님)                                                                |
| <span style="color:blue">**REL_RWY_R**</span>        | 도로 상 사고 여부 (1=도로 위에서 발생, 0=아님)                                                              |
| <span style="color:blue">**PROFIL_I_R**</span>       | 도로의 기울기 상태 (1=평지, 0=기타)                                                                         |
| <span style="color:blue">**SPD_LIM**</span>          | 제한 속도 (마일 단위)                                                                                       |
| <span style="color:blue">**SUR_CON**</span>          | 도로 표면 상태 (1=건조, 2=습기, 3=눈/진흙, 4=얼음, 5=모래/흙/기름, 8=기타, 9=알 수 없음)                   |
| <span style="color:blue">**TRAF_CON_R**</span>       | 교통 제어 장치 (0=없음, 1=신호등, 2=기타 (표지판, 교통 경찰 등))                                           |
| <span style="color:blue">**TRAF_WAY**</span>         | 교통 방향 (1=양방향, 2=분리된 고속도로, 3=일방통행)                                                          |
| VEH_INVL         | 관련 차량 수                                                                                                |
| <span style="color:blue">**WEATHER_R**</span>        | 날씨 조건 (1=악천후 없음, 2=비/눈 또는 기타 악천후)                                                         |
| INJURY_CRASH     | 부상 사고 여부 (1=예, 0=아니오)                                                                            |
| NO_INJ_I         | 부상자 수                                                                                                   |
| PRPTYDMG_CRASH   | 재산 피해 여부 (1=재산 피해 있음, 2=재산 피해 없음)                                                         |
| FATALITIES       | 사망자 여부 (1=사망 있음, 0=사망 없음)                                                                      |
| MAX_SEV_IR       | 사고 심각도 (0=부상 없음, 1=비치명적 부상, 2=치명적 부상)                                                   |

## ii.
관련된 예측 변수들(과 결과 변수에 해당하는 INJURY)로 구성된 완전한 학습 데이터셋을 이용해 나이브 베이즈 분류기를 실행하시오. 모든 예측 변수가 범주형이라는 점에 주목하고, 정오 행렬을 나타내시오.

In [25]:
import pandas as pd
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix, classification_report

# Define the variables to be used
vars = ["INJURY", "HOUR_I_R", "ALIGN_I", "WRK_ZONE", "WKDY_I_R",
        "INT_HWY", "LGTCON_I_R", "PROFIL_I_R", "SPD_LIM", "SUR_COND",
        "TRAF_CON_R", "TRAF_WAY", "WEATHER_R"]

# Assuming train_df is your training DataFrame
X = train_df[vars[1:]]  # Features (excluding the target variable)
y = train_df['INJURY'].astype('category')  # Target variable

# Create and fit the Naive Bayes model
nb_total = CategoricalNB(alpha = 0, fit_prior = False, force_alpha = True)
nb_total.fit(X, y)

# Generate predictions
y_pred = nb_total.predict(X)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y, 
y_pred, 
labels=["Yes", "No"]
)

# Print the confusion matrix and classification report
print("Confusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y, y_pred, target_names=["no", "yes"]))

acc_score = accuracy_score(y, y_pred)
print("\nAccuracy Score about Training data: ", (acc_score))

error_rate = 1 - acc_score
print("\nError rate about Training data: ", error_rate)


Confusion Matrix:
[[7911 4928]
 [6745 5725]]

Classification Report:
              precision    recall  f1-score   support

          no       0.54      0.46      0.50     12470
         yes       0.54      0.62      0.58     12839

    accuracy                           0.54     25309
   macro avg       0.54      0.54      0.54     25309
weighted avg       0.54      0.54      0.54     25309


Accuracy Score about Training data:  0.5387806709075823

Error rate about Training data:  0.4612193290924177


## iii. 
검증 데이터셋에 대한 전체 오차는 얼마인가?

In [26]:
X_valid = valid_df[vars[1:]]  # Features (excluding the target variable)
y_valid = valid_df['INJURY'].astype('category')  # Target variable

# Generate predictions
y_pred_valid = nb_total.predict(X_valid)

# Generate the confusion matrix
conf_matrix_valid = confusion_matrix(y_valid, 
y_pred_valid, 
labels=["Yes", "No"]
)

# Print the confusion matrix and classification report
print("Confusion Matrix about Valid data:")
print(conf_matrix_valid)

print("\nClassification Report about Valid data:")
print(classification_report(y_valid, y_pred_valid, target_names=["No", "Yes"]))

acc_score_valid = accuracy_score(y_valid, y_pred_valid)
print("\nAccuracy Score about valid data: ", (acc_score_valid))

error_rate_valid = 1 - acc_score_valid
print("\nError rate about valid data: ", error_rate_valid)


Confusion Matrix about Valid data:
[[5270 3353]
 [4437 3814]]

Classification Report about Valid data:
              precision    recall  f1-score   support

          No       0.53      0.46      0.49      8251
         Yes       0.54      0.61      0.58      8623

    accuracy                           0.54     16874
   macro avg       0.54      0.54      0.53     16874
weighted avg       0.54      0.54      0.54     16874


Accuracy Score about valid data:  0.5383430129192841

Error rate about valid data:  0.4616569870807159


## iv. 
(검증 데이터셋을 사용한) 나이브 규칙과 비교한 상대적인 퍼센트 향상 정도는 얼마인가?

In [27]:
import numpy as np



imp = error_rate_valid - error_rate

# Calculate the percent improvement
percent_improvement = imp * 100  # Convert to percentage

# Print the result
print(f"The percent improvement is {percent_improvement:.2f}%")


The percent improvement is 0.04%


## v.
피벗 테이블의 조건부 확률들을 검토하시오. P(INJURY = No | SPD_LIM = 5)에 대해 확률값이 0이 되는 이유는 무엇인가?

In [28]:
import pandas as pd
import numpy as np

# Calculate class prior probabilities manually
class_counts = y.value_counts(normalize=True)  # Normalize to get probabilities
print("A-priori probabilities:")
print("## Y")
print("##", " ".join(class_counts.index))  # Output class labels
print("##", " ".join([f"{count:.2f}" for count in class_counts.values]), "\n")

# Conditional probabilities
print("## Conditional probabilities:")
for feature in X.columns:
    print(f"##      {feature}")
    
    # Calculate conditional probabilities for each feature value given each class
    feature_prob = pd.crosstab(X[feature], y, normalize='columns')
    
    # Get the unique values for this feature (sorted for consistent order)
    feature_values = feature_prob.index
    print("##   Y", "  ","   ".join(map(str, feature_values)))
    
    # Print conditional probabilities for each class
    for class_label in class_counts.index:
        print(f"##   {class_label} ", end="")
        for value in feature_values:
            prob = feature_prob.at[value, class_label] if value in feature_prob.index else 0
            print(f"{prob:.2f} ", end="")
        print()  # Newline for next class
    print()  # Newline for next feature


A-priori probabilities:
## Y
## Yes No
## 0.51 0.49 

## Conditional probabilities:
##      HOUR_I_R
##   Y    0   1
##   Yes 0.57 0.43 
##   No 0.57 0.43 

##      ALIGN_I
##   Y    1   2
##   Yes 0.86 0.14 
##   No 0.87 0.13 

##      WRK_ZONE
##   Y    0   1
##   Yes 0.98 0.02 
##   No 0.98 0.02 

##      WKDY_I_R
##   Y    0   1
##   Yes 0.24 0.76 
##   No 0.22 0.78 

##      INT_HWY
##   Y    0   1   9
##   Yes 0.87 0.13 0.00 
##   No 0.85 0.15 0.00 

##      LGTCON_I_R
##   Y    1   2   3
##   Yes 0.70 0.11 0.19 
##   No 0.69 0.12 0.18 

##      PROFIL_I_R
##   Y    0   1
##   Yes 0.76 0.24 
##   No 0.75 0.25 

##      SPD_LIM
##   Y    5   10   15   20   25   30   35   40   45   50   55   60   65   70   75
##   Yes 0.00 0.00 0.00 0.00 0.10 0.09 0.21 0.11 0.15 0.04 0.15 0.04 0.06 0.03 0.01 
##   No 0.00 0.00 0.00 0.01 0.10 0.09 0.20 0.09 0.16 0.04 0.16 0.04 0.06 0.04 0.01 

##      SUR_COND
##   Y    1   2   3   4   9
##   Yes 0.82 0.15 0.01 0.01 0.00 
##   No 0.78 0.18 0.02 0.03

In [29]:
# SPD_LIM이 5인 사례 확인
spd_lim_5_cases = train_df[train_df['SPD_LIM'] == 5]

# 결과 출력
if spd_lim_5_cases.empty:
    print("데이터셋에 SPD_LIM = 5인 교통사고 사례가 존재하지 않습니다.")
else:
    print("데이터셋에 SPD_LIM = 5인 교통사고 사례가 존재합니다:")
    print(spd_lim_5_cases)


데이터셋에 SPD_LIM = 5인 교통사고 사례가 존재합니다:
      HOUR_I_R ALCHL_I ALIGN_I STRATUM_R WRK_ZONE WKDY_I_R INT_HWY LGTCON_I_R  \
9827         0       2       1         0        0        0       0          2   
24185        0       2       2         0        0        0       0          1   
33330        1       1       1         1        0        1       0          1   
11847        0       1       1         0        0        1       0          2   
40366        0       2       1         1        0        1       0          1   

      MANCOL_I_R PED_ACC_R  ... TRAF_CON_R TRAF_WAY VEH_INVL WEATHER_R  \
9827           0         0  ...          0        1        1         1   
24185          2         0  ...          2        3        2         1   
33330          2         0  ...          1        1        2         1   
11847          0         1  ...          2        1        1         1   
40366          0         0  ...          0        1        1         1   

      INJURY_CRASH NO_INJ_I PRPTY

### ANS)
P(INJURY = No | SPD_LIM = 5)에서 확률 값이 0이 되는 이유는, SPD_LIM = 5인 사례가 매우 적고, 그 중 INJURY = No로 분류된 사례가 충분하지 않기 때문입니다.

훈련 데이터에서 SPD_LIM = 5인 경우는 총 5개이며, 그중 INJURY = No로 나타난 사례는 단 2개에 불과합니다. 이처럼 제한된 데이터로 인해 확률 추정에서 P(INJURY = No | SPD_LIM = 5)가 0으로 계산된 것입니다.